In [1]:
import numpy as np
import kirchhoff.circuit_init as kfi
from hailhydro.flow_init import Flow

from goflow.adapter import init_ivp as gi
from plotly_gif import GIF, capture
import plotly.graph_objects as go

#initialize circuit+flow pattern
# C = kfi.initialize_circuit_from_crystal('laves',3)
C = kfi.initialize_circuit_from_crystal('triagonal_planar',3)

pars_src = {
    'modesSRC': 'root_geometric'
#     'modeSRC': 'root_short',
#     'modeSRC': 'dipole_border',
}
pars_plx = {
    'modePLX':'default',
}
# set model and model parameters
pars_model = {
    'alpha_0':1.,
    'alpha_1':1.
}
# # initialize dynamic system and set integration parameters
morpheus = gi.morph_dynamic(C, 'murray', [pars_model, pars_src, pars_plx]) 
morpheus.evals = 200

Set source: default
set_source_landscape(): default is set and consistent :)
Set plexus: default
set_plexus_landscape(): default is set and consistent :)


In [2]:
# numerically evaluate the system
cnd = morpheus.flow.circuit.edges['conductivity']
cnd_scale = morpheus.flow.circuit.scales['conductance']

sp = {
    't0': 1e-05,
    't1': 4.,
    'x0': np.power(cnd/cnd_scale,0.25)*0.1,
}
nsol = morpheus.nlogSolve((sp['t0'],sp['t1']), sp['x0'])
# print(nsol)

dataPoints = zip(nsol.t,nsol.y.transpose())
murrayModel = morpheus.model
args = murrayModel.solver_options['args']
cost = [murrayModel.calc_cost_stimuli(t, y, *args)[0] for t, y in dataPoints]

In [3]:
aux = {
    'color_nodes': ['#030512'],
    'colormap': ['RdPu'],
}
# plot the network plexus and attributes
# plexus
morpheus.flow.circuit.edges['conductivity'] = np.round(nsol.y.transpose()[0],5)
lw = morpheus.flow.circuit.edges['conductivity']
morpheus.flow.circuit.edges['flow_rate'], dP = morpheus.flow.calc_configuration_flow()
morpheus.flow.circuit.edges['flow_rate'] = np.round(np.absolute(morpheus.flow.circuit.edges['flow_rate']),5)
fig = morpheus.flow.circuit.plot_circuit(linewidth = [10.*lw], color_edges = [lw], **aux)
fig.write_html("../docs/plexus.html")

# final
morpheus.flow.circuit.edges['conductivity'] = np.round(nsol.y.transpose()[-1],5)
lw = morpheus.flow.circuit.edges['conductivity']
morpheus.flow.circuit.edges['flow_rate'], dP = morpheus.flow.calc_configuration_flow()
morpheus.flow.circuit.edges['flow_rate'] = np.round(np.absolute(morpheus.flow.circuit.edges['flow_rate']),5)
fw = morpheus.flow.circuit.edges['flow_rate']

fig = morpheus.flow.circuit.plot_circuit(linewidth = [5.*lw], color_edges = [lw], **aux)
fig.write_html("../docs/final.html")

fig = morpheus.flow.circuit.plot_circuit(linewidth = [5.*fw], color_edges = [fw])
fig.write_html("../docs/final_flow.html")

In [4]:
# plot the radii dynamics
fig = go.Figure()
for i, ys in enumerate(nsol.y):
    
    e = morpheus.flow.circuit.edges['label'][i]
    fig.add_trace(
        go.Scatter(
            x= nsol.t, 
            y= ys, 
            mode='lines',
            line=dict(color='rgba(0,0,255,0.1)'),
            name=f'vessel {e}'
        )
    )
fig.update_layout( dict(
#     showlegend =  False,
    )
)
fig.update_xaxes(type="log")
# fig.update_yaxes(range=[y_min, y_max]) # linear range
fig.write_html("../docs/dynamics.html")